# Dashboard weather analyst

## Import libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import numpy as np
from scipy import stats
import param
import panel as pn
pn.extension('tabulator')
pn.extension('plotly')

import hvplot.pandas
import plotly.express as px
# hvplot.extension('plotly')

## Read data

In [3]:
df = pd.read_csv('data/dashboard_data.csv')
df = df.drop(columns='Unnamed: 0')
df.sample(10)

,Date,Amplitude,Time,Temperature,Dew_Point,Humidity,Wind,Wind_Speed,Pressure,Condition,Region,Month,Year
194419,2022-09-23,11.111111,11:30 AM,32.222222,22.222222,55.0,N,6.0,29.81,Fair,Ha Noi,9,2022
74811,2020-06-08,12.777778,6:00 AM,30.000000,23.888889,70.0,NNW,3.0,29.67,Fair,Da Nang,6,2020
60814,2020-03-02,13.888889,8:30 PM,25.000000,22.777778,89.0,SE,7.0,29.94,Partly Cloudy,Da Nang,3,2020
73645,2020-05-31,8.888889,5:30 PM,31.111111,26.111111,75.0,S,8.0,29.73,Partly Cloudy,Sai Gon,5,2020
139287,2021-09-04,8.888889,11:30 AM,32.777778,22.777778,55.0,CALM,0.0,29.70,Mostly Cloudy,Da Nang,9,2021
158476,2022-01-16,17.777778,9:00 PM,22.777778,20.000000,83.0,WNW,2.0,30.00,Fair,Da Nang,1,2022
96898,2020-11-11,12.222222,3:30 PM,26.111111,22.777778,83.0,ENE,13.0,29.88,Light Rain,Da Nang,11,2020
132733,2021-07-20,12.222222,7:30 AM,27.777778,27.222222,94.0,NW,3.0,29.58,Mostly Cloudy,Ha Noi,7,2021
131874,2021-07-14,10.000000,10:00 AM,32.777778,22.222222,52.0,SE,10.0,29.69,Fair,Ha Noi,7,2021
33218,2019-08-23,8.888889,12:00 AM,27.222222,25.000000,89.0,WSW,6.0,29.79,Partly Cloudy,Sai Gon,8,2019


In [4]:
df.columns

Index(['Date', 'Amplitude', 'Time', 'Temperature', 'Dew_Point', 'Humidity',
       'Wind', 'Wind_Speed', 'Pressure', 'Condition', 'Region', 'Month',
       'Year'],
      dtype='object')

## Create DashBoard

In [5]:
# set(df['year'])
Years = list(set(df['Year']))
Regions = list(set(df['Region']))
inputs = ['Month','Temperature','Dew_Point', 'Humidity','Amplitude','Wind_Speed', 'Pressure']

class Dashboard(param.Parameterized):
    Region = param.Selector(Regions, default= Regions[1])
    select = param.Selector(Years, default= Years[1])
    X_variable = param.Selector(inputs, default=inputs[2])
    Y_variable = param.Selector(inputs, default=inputs[1])
    
    @param.depends('X_variable', 'Y_variable', 'select', 'Region')
    def scatter_plot(self):
        dfa = df[df['Region']==self.Region]
        return  dfa[dfa['Year']== self.select].hvplot.scatter(x=self.X_variable, y=self.Y_variable, width=500, height=450)
    
    @param.depends('Y_variable', 'select', 'Region')
    def line_plot(self):
        dfa = df[df['Region']==self.Region]
        return  dfa[dfa['Year']== self.select].hvplot.line('Date', self.Y_variable, width=1500)
    
    @param.depends('Y_variable', 'select', 'Region')
    def violin_plot(self):
        dfa = df[df['Region']==self.Region]
        return  dfa[dfa['Year']== self.select].hvplot.violin(self.Y_variable, 'Month', width=1500)
    
    # Subplot
    
    @param.depends('Y_variable', 'select', 'Region')
    def subline_plot_1(self):
        dfa = df[df['Region']==self.Region]
        return  dfa[dfa['Year']== self.select].groupby('Month').mean().hvplot(x='Month', y=['Temperature', 'Amplitude'],
                 value_label='Mean', subplots=True, width=750)
    
    @param.depends('Y_variable', 'select', 'Region')
    def subline_plot_2(self):
        dfa = df[df['Region']==self.Region]
        return  dfa[dfa['Year']== self.select].groupby('Month').mean().hvplot(x='Month', y=['Dew_Point', 'Humidity', 'Wind_Speed'],
                 value_label='Mean', subplots=True, width=500)
    
    @param.depends('Y_variable', 'select', 'Region')
    def box_plot(self):
        dfa = df[df['Region']==self.Region]
        df_box = dfa[dfa['Year']== self.select].groupby('Date').max()
        return df_box.hvplot.box(by='Month', y= self.Y_variable, legend=False, width=500)
    
    @param.depends('Y_variable', 'select', 'Region')
    def pie_plot(self):
        dfb = df[df['Region']==self.Region]
        dfa = pd.DataFrame(dfb[dfb['Year']== self.select]['Wind'].value_counts())
        fig = px.pie(dfa,values='Wind', names=dfa.index)
        return pn.pane.Plotly(fig, config={'responsive': True} , width=500)
    
    @param.depends('Y_variable', 'select', 'Region')
    def barh_condition(self):
        dfa = df[df['Region']==self.Region]
        dfc = pd.DataFrame(dfa[dfa['Year']== self.select].groupby('Condition').count()).sort_values('Time')[10:]
        return dfc.hvplot.barh(y='Time', width=500)
    
    @param.depends('select', 'Region')
    def amp_plot(self):
        dfa = df[df['Region']==self.Region]
        amp = pd.DataFrame(dfa[dfa['Year']== self.select].groupby("Date")['Temperature'].max() - dfa[dfa['Year']== self.select].groupby("Date")['Temperature'].min())
        amp.columns = ['Amplitude']
        return amp.hvplot.line('Date', self.Y_variable, width=1500)
        # fig = px.line(amp, x=amp.index,y='Amplitude', title='Hướng gió')
        # return pn.pane.Plotly(fig, config={'responsive': True}, width=1500)
    
    @param.depends('select', 'Region')
    def heat_map(self):
        dfa = df[df['Region'] == self.Region]
        z =dfa[dfa['Year']== self.select][['Temperature', 'Dew_Point', 'Humidity', 'Pressure']].corr()
        fig = px.imshow(z, text_auto=True,  color_continuous_scale='Blues')
        return pn.pane.Plotly(fig, config={'responsive': True} , width=500)
    
    @param.depends('select', 'Region')
    def win_speed(self):
        dfa = df[df['Region']==self.Region]
        wind = dfa[dfa['Year']== self.select].groupby('Wind_Speed').count()
        return wind.hvplot.bar(y='Time', width=500)
            
        
         
        
    
    def panel(self):
        return pn.Row(
            pn.Param(self, width=200, name = "Plot Settings", sizing_mode="fixed"),
            pn.Column(
                 # self.line_plot,
                self.subline_plot_1,
                self.subline_plot_2,
                pn.Row(self.heat_map,self.pie_plot, self.scatter_plot),
                pn.Row(self.win_speed, self.box_plot,self.barh_condition),
            # self.amp_plot,
            # self.subline_plot,
            # self.box_plot,
            # self.pie_plot,
            # self.scatter_plot,
            self.violin_plot
            # self.line_plot,
            # self.barh_condition,
            # self.heat_map,
            # self.win_speed,self.barh_condition
            )
        )
dashboard = Dashboard(name='Weather_Dashboard')
component = dashboard.panel()
component

Row
    [0] Param(Dashboard, name='Plot Settings', sizing_mode='fixed', width=200)
    [1] Column
        [0] ParamMethod(method)
        [1] ParamMethod(method)
        [2] Row
            [0] ParamMethod(method)
            [1] ParamMethod(method)
            [2] ParamMethod(method)
        [3] Row
            [0] ParamMethod(method)
            [1] ParamMethod(method)
            [2] ParamMethod(method)
        [4] ParamMethod(method)

In [6]:
pn.template.FastListTemplate(site="Panel", title="Weather_Dashboard", 
                             main=[
                                 component 
                             ]).servable();

## Run in terminal

In [ ]:
# bokeh serve --show --port 4007 .\Dashboard.ipynb
